# 多头注意力（Multi-Head Attention）

多头注意力（Multi-Head Attention）是一种在自然语言处理（NLP）任务中广泛使用的机制，特别是在 Transformer 模型中。多头注意力机制通过并行计算多个注意力头（Attention Heads），每个头关注输入序列的不同部分，从而捕捉更丰富的信息。

## 1. 背景

在传统的注意力机制中，模型通常只计算一个注意力头，这可能导致模型在捕捉输入序列的复杂关系时表现不佳。为了解决这个问题，多头注意力机制被引入，允许模型在多个不同的子空间中并行计算注意力，从而捕捉更丰富的信息。

## 2. 多头注意力的核心思想

多头注意力的核心思想是通过并行计算多个注意力头，每个头关注输入序列的不同部分。具体来说，多头注意力机制将输入序列的嵌入向量分成多个子空间，每个子空间对应一个注意力头。每个注意力头独立计算注意力分数，并生成一个加权上下文向量。最后，将所有注意力头的加权上下文向量拼接起来，并通过一个线性变换，得到最终的加权上下文向量。

## 3. 工作原理

### 3.1 线性变换

首先，对输入序列进行线性变换，得到多个查询（Query）、键（Key）和值（Value）的投影。

- **公式**：
  \[
  Q_i = XW_i^Q, \quad K_i = XW_i^K, \quad V_i = XW_i^V
  \]
  其中，\( X \) 是输入序列的嵌入向量，\( W_i^Q \)、\( W_i^K \) 和 \( W_i^V \) 是第 \( i \) 个注意力头的线性变换矩阵。

### 3.2 并行计算

对每个注意力头，分别计算注意力分数、归一化和加权求和，得到多个加权上下文向量。

- **公式**：
  \[
  \text{score}(Q_i, K_i) = \frac{Q_i \cdot K_i^T}{\sqrt{d_k}}
  \]
  \[
  \alpha_i = \text{softmax}(\text{score}(Q_i, K_i))
  \]
  \[
  \text{context}_i = \alpha_i \cdot V_i
  \]
  其中，\( Q_i \)、\( K_i \) 和 \( V_i \) 是第 \( i \) 个注意力头的查询、键和值向量，\( d_k \) 是键向量的维度。

### 3.3 拼接与线性变换

将多个加权上下文向量拼接起来，并通过一个线性变换，得到最终的加权上下文向量。

- **公式**：
  \[
  \text{context} = \text{concat}(\text{context}_1, \text{context}_2, \dots, \text{context}_h)W^O
  \]
  其中，\( h \) 是注意力头的数量，\( W^O \) 是输出线性变换矩阵。

![MHA](https://zh-v2.d2l.ai/_images/multi-head-attention.svg "MHA")

## 4. 优点与局限性

### 4.1 优点

- **并行计算**：多头注意力机制允许模型在多个不同的子空间中并行计算注意力，从而捕捉更丰富的信息。
- **捕捉复杂关系**：通过多个注意力头，模型可以捕捉输入序列中的复杂关系，提高模型的性能。

### 4.2 局限性

- **计算复杂度**：多头注意力机制的计算复杂度较高，尤其是在处理长序列时。
- **可解释性**：虽然多头注意力机制提高了模型的性能，但它也使得模型的可解释性降低，因为注意力权重是动态计算的，难以直观理解。

## 5. 应用场景

- **机器翻译**：在生成翻译后的句子时，多头注意力机制可以帮助模型关注源语言句子中的重要部分。
- **文本摘要**：在生成摘要时，多头注意力机制可以帮助模型关注原文中的关键信息。
- **问答系统**：在生成回答时，多头注意力机制可以帮助模型关注问题中的关键部分和相关上下文。

## 6. 总结

多头注意力机制是一种在 Transformer 模型中广泛使用的机制，通过并行计算多个注意力头，捕捉输入序列中的复杂关系，从而提高模型的性能。尽管计算复杂度较高，但多头注意力机制在许多 NLP 任务中表现出色，成为现代深度学习模型的核心组件之一。

## 简单代码实现

In [ ]:
#@save
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    def forward(self, queries, keys, values, valid_lens):
        # queries，keys，values的形状:
        # (batch_size，查询或者“键－值”对的个数，num_hiddens)
        # valid_lens　的形状:
        # (batch_size，)或(batch_size，查询的个数)
        # 经过变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者“键－值”对的个数，
        # num_hiddens/num_heads)
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)

        if valid_lens is not None:
            # 在轴0，将第一项（标量或者矢量）复制num_heads次，
            # 然后如此复制第二项，然后诸如此类。
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0)

        # output的形状:(batch_size*num_heads，查询的个数，
        # num_hiddens/num_heads)
        output = self.attention(queries, keys, values, valid_lens)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

为了能够使多个头并行计算， 上面的MultiHeadAttention类将使用下面定义的两个转置函数。 具体来说，transpose_output函数反转了transpose_qkv函数的操作。



In [ ]:
#@save
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，
    # num_hiddens/num_heads)
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)

    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)

    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])


#@save
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)